In [1]:
using LinearAlgebra, Distributions, Statistics, Plots, KernelFunctions, Random, Interpolations, ForwardDiff, Gridap, NPZ, ProgressMeter
import Gridap: ∇

In [2]:
using IJulia

# Elliptic ODE with parametrised diffusion coefficient:

$\newcommand{R}{\mathbb{R}}$

\begin{align}
-\frac{\mathrm{d}}{\mathrm{d}x}\left(\kappa(x;p)\frac{\mathrm{d}}{\mathrm{d}x} u(x;p)\right) &= f(x), \quad x\in\Omega:=[0,1] \\
u(x) &= 0, \quad x\in\partial\Omega
\end{align}

where: $\kappa:\R\times\R^{d}\rightarrow\R$ is defined by:

$$
\log\kappa(x;p) := \sum_{k=1}^{d} p_{k} \lambda_{k} \phi_{k}(x)
$$

where the the pairs $\{\lambda_{k},\phi_{k}(x)\}$ are the eigenpairs of the correlation operator associated with the Matern-1/2 covariance function:

$$
c(x,y) = \exp\left(-\beta^{-1}|x-y|\right)
$$

where we truncate the KL-expansion at $d=100$ terms so our parameter vector $p\in\R^{100}$.

Note here the $\{p_{k}\}_{k=1}^{d}$ are i.i.d. $\mathcal{N}(0,1)$ random variables

In [3]:
function κ(x, p, β)
    d = length(p)
    xs = range(0, stop=1, length=d)
    
    # form kernel and kernel mat
    c = with_lengthscale(Matern12Kernel(), β)
    K = kernelmatrix(c, xs)
    
    # get spectrum
    spectrum = eigen(K, sortby = (x -> -real(x)))
    
    # form eigenpairs
    Φ = √(d) * spectrum.vectors
    ϕ(x::Float64,i::Int) = linear_interpolation(xs, Φ[:,i])(x)
    λ = spectrum.values / d
    
    # compute log_κ and κ
    log_κ = sum(p .* λ .* ϕ.(x, 1:d))
    return exp(log_κ)
end

κ (generic function with 1 method)

In [17]:
Random.seed!(42)
d = 100
p = rand(Normal(), d)
β = 0.5
κ(0.5, p, β)

1.287367884787411

In [18]:
function dκdp(x,p,β,j)
    d = length(p)
    xs = range(0, stop=1, length=d)
    
    # form kernel and kernel mat
    c = with_lengthscale(Matern12Kernel(), β)
    K = kernelmatrix(c, xs)
    
    # get spectrum
    spectrum = eigen(K, sortby = (x -> -real(x)))
    
    # form eigenpairs
    Φ = √(d) * spectrum.vectors
    ϕ(x::Float64,i::Int) = linear_interpolation(xs, Φ[:,i])(x)
    λ = spectrum.values / d
    
    log_κ = sum(p .* λ .* ϕ.(x, 1:d))
    return λ[j] * ϕ(x,j) * exp(log_κ)
end

dκdp (generic function with 1 method)

In [19]:
function dudp(p, β, κ, dκdp, x0, f, n=50, order=1)
    # set up mesh and spaces
    domain = (0,1)
    partition = (n)
    model = CartesianDiscreteModel(domain, partition)

    reffe = ReferenceFE(lagrangian,Float64,order)
    V0 = TestFESpace(model,reffe,conformity=:H1,dirichlet_tags="boundary")
    U = TrialFESpace(V0,x->0)

    degree = order + 1

    # solve for FEM solution uh of original problem (forward pass)
    Ω = Triangulation(model)
    dΩ = Measure(Ω,degree)

    kappa(x::VectorValue) = κ(x[1],p, β)
    a(u,v) = ∫( kappa * ∇(v)⊙∇(u) )*dΩ
    b(v) = ∫( v*f )*dΩ

    op = AffineFEOperator(a,b,U,V0)
    uh = solve(op)

    # solve for adjoint variable λ (back pass)
    δ = DiracDelta(model, Point(x0))
    b_λ(v) = -1*δ(v)
    
    op_λ = AffineFEOperator(a,b_λ,U,V0)
    λ = solve(op_λ)

    # compute derivative for each parameter and return and return
    grads = []
    d = length(p)
    for j ∈ 1:d
        dkappa(x::VectorValue) = dκdp(x[1],p,β,j)
        sens = sum( ∫( dkappa * ∇(λ)⋅∇(uh) )*dΩ )
        push!(grads, sens)
    end
    
    return grads
end

dudp (generic function with 3 methods)

In [27]:
f(x) = 1.0
x0 = 0.5
Random.seed!(42)
d = 30
p = rand(Normal(), d)
β = 1

1

In [28]:
@time begin
    grads = dudp(p, β, κ, dκdp, x0, f);
end

  6.172894 seconds (4.56 M allocations: 658.048 MiB, 4.90% gc time, 70.98% compilation time: 3% of which was recompilation)


30-element Vector{Any}:
  0.0678212426839818
 -0.00037901807764259923
  0.0025896535232198385
  2.8343032973214812e-5
  8.142115401200362e-5
 -3.541639758198761e-6
 -9.55969258036967e-6
 -5.310497714935109e-6
  2.4808146637444575e-5
 -3.9320637013518807e-7
  1.2215346394873323e-5
  7.551970634231023e-7
  1.2244277378999908e-5
  ⋮
  4.608697794179263e-6
  1.4265435928756865e-7
 -3.942122716293574e-6
  8.487900656810584e-8
  2.6938953548187514e-6
 -5.310339328182543e-8
  2.360265103627797e-6
 -6.648034382215336e-8
 -1.620642753210826e-6
 -6.020474228647302e-8
 -4.643721144213752e-7
  7.09305485558147e-8

In [8]:
function getAS(x0, ps, β, κ, dκdp, f)
    # compute gradients at each p in ps
    Grads = map(p -> dudp(p, β, κ, dκdp, x0, f), eachcol(ps))

    # take outer product of each grad with itself
    C_x = map(grad -> grad * grad', Grads)

    # compute mean of outer products
    C_hat = Statistics.mean(C_x)

    # eigen decomposition of C_hat
    spectrum = eigen(C_hat)
    λ = spectrum.values

    # compute proportion of spectrum explained by max eigenvector
    λ_max = max(λ...)
    prop = λ_max / sum(abs,λ)

    # get AS as leading eigenvector
    W = spectrum.vectors[:, argmax(λ)]
    @assert W' * W ≈ 1.0

    return W, λ, prop
end

getAS (generic function with 1 method)

In [15]:
@time begin
    getAS(0.5,ps,β,κ,dκdp,f)
end

 45.868338 seconds (14.18 M allocations: 15.627 GiB, 4.43% gc time)


([-0.9961676170319284, -0.02249621187019473, -0.08437087606066936, 0.004208940153565632, 0.0026203008860199244, -0.00040067917459817866, 0.00048071555468345785, -0.00037096308012848596, 0.0005098084139979852, 0.00015714630649307497  …  1.259025576937096e-5, 1.299682665231507e-6, -9.686464024918641e-6, 1.329056495599523e-6, -7.221111896033929e-6, -1.179959269450559e-6, -4.444629106794739e-6, 1.1336938703309974e-6, -2.150575319159742e-6, 1.1392145639464755e-6], [-5.32655293607305e-18, -5.294350290005522e-18, -4.6285562548794024e-18, -3.170152093818658e-18, -2.803518498362368e-18, -2.6209496607549606e-18, -5.557469523298945e-19, -1.8347735847569123e-19, -7.14847226070694e-20, -1.0254507737904766e-20  …  1.6304605513933722e-12, 3.983100024430367e-12, 1.4583906085832793e-11, 1.5099758318264575e-10, 1.3572472779753037e-9, 4.888540177047233e-9, 2.7682487008978485e-7, 5.425569646212399e-7, 7.442010994547833e-6, 0.014164579612265488], 0.9994166444070419)

In [10]:
for (i,x) in enumerate(xs)
    IJulia.clear_output(true)
    @show i
end

LoadError: UndefVarError: `xs` not defined

In [11]:
β = 1.0

1.0

In [34]:
(((6*100)/60)*5)

50.0

In [37]:
Random.seed!(42)
d = 30
M = 100
ps = rand(Normal(), (d,M));

In [38]:
xs = range(0.01,stop=0.99,length=5)

0.01:0.245:0.99

In [ ]:
AS_results = []
for (i,x) ∈ enumerate(xs)
    IJulia.clear_output(true)
    @show i
    result = getAS(x, ps, β, κ, dκdp, f)
    push!(AS_results, result)
end

i = 3


In [40]:
# AS_results = @showprogress map(x -> getAS(x, ps, β, κ, dκdp, f), xs);